# 0. Libraries

In [8]:
import requests
import pandas as pd
import json
import joblib

# 1. API Call - Dados de Vacinação

## 1.1. Funções

In [18]:
def getAPIData(pages,
    username = 'imunizacao_public',
    password = 'qlto5t&7r_@+#Tlstigi',
    url_firstpage = 'https://imunizacao-es.saude.gov.br/_search?scroll=1m',
    url_nextpages = 'https://imunizacao-es.saude.gov.br/_search/scroll'
    ):

    '''
    Função para extrair todo os dados de vacinação da API.
    Para mais detalhes em relação à execução da chamada da API confira o manual contido dentro da pasta others.

    A função retorna o dataframe completo mais o status das suas respectivas respostas de consulta.
    Consulta pode demorar bastante, tempo estimado de 10 páginas por minuto, dependendo do processador.
    '''

    responses = []
    scroll_ids = []

    # Chamada da primeira página.
    response_pag1 = requests.post(url_firstpage, auth=(username, password), json={"size": 10000})
    
    print(f'Page 1 - Status Response: {response_pag1.status_code}')

    responses.append(response_pag1.status_code)

    # Normalização do arquivo em json e tranferência para um Dataframe.
    data = response_pag1.json()
    df = pd.json_normalize(data, record_path=['hits','hits'])

    # Para as próximas chamadas devemos gerar o scroll id para inserir no corpo da chamada da API
    scroll_id = data['_scroll_id']

    items = df.shape[0]

    # Loop while para conferir os conteúdos das demais páginas onde pag representa o número da página
    pag = 1
    while items == 10000 and pag < pages:

        pag+=1

        try:
            # Chamada das demais páginas.
            response_pagN = requests.post(url_nextpages, auth=(username, password), json={"scroll_id":scroll_id,"scroll": "1m"})
    
            if pag%10 == 0:
                print(f'Page {pag} - Status Response: {response_pagN.status_code}')
    
            responses.append(response_pagN.status_code)
    
            # Normalização do arquivo em json e tranferência para um Dataframe.
            dataN = response_pagN.json()
            dfN = pd.json_normalize(dataN, record_path=['hits','hits'])
    
            # Atribuição ao novo valor de items para verificação se a página contém mais do que o número máximo de registros (10000)
            items = dfN.shape[0]
    
            # Concatenação dos dados extraídos das seguintes leituras da API
            df = pd.concat([df,dfN],ignore_index=True)

            scroll_id = dataN['_scroll_id']

        except MemoryError:
            print(f'At Page {pag} there was not enough memory to execute the task - Status Response: {responses[-1]}\nThe scroll_id has been added to the list to a future attempt')
            scroll_ids.append(scroll_id)

        except ConnectionError:
            print('No Internet Connection to Perform the API Call. The scroll_id has been added to the list to a future attempt')
            scroll_ids.append(scroll_id)


    print(f'{pag} Pages have been loaded successfully!')

    print(f'The following responses status have been retrieved: {set(responses)}')

    print(f'Final Dataset Shape:\nRows: {df.shape[0]}\nColumns: {df.shape[1]}')
    
    return df, responses, scroll_ids
        

In [55]:
def storeData(df, filename):
    
    filename = str(filename)

    try:
        joblib.dump(df, f'../data/{filename}.pkl')
        print(f'File {filename}.pkl has been stored successfully in data folder')
    except:
        print('You must give a valid format for the filename')

In [54]:
def loadData(filename):

    filename = str(filename)

    try:
        df = joblib.load(f'../data/{filename}.pkl')
        print(f'File {filename}.pkl has been loaded successfully from data folder')
        return df
    except:
        print('You must give a valid format for the filename')

In [53]:
def updateAPIData(filename,
    username = 'imunizacao_public',
    password = 'qlto5t&7r_@+#Tlstigi',
    url_firstpage = 'https://imunizacao-es.saude.gov.br/_search?scroll=1m',
    url_nextpages = 'https://imunizacao-es.saude.gov.br/_search/scroll'
    ):

    filename = str(filename)

    df = loadData(filename)

    lastRecord = df['_source.data_importacao_rnds'][0][:10]

    responses = []
    scroll_ids = []

    # Chamada da primeira página.
    response_pag1 = requests.post(url_firstpage, auth=(username, password), json={"size": 10000})
    
    print(f'Page 1 - Status Response: {response_pag1.status_code}')

    responses.append(response_pag1.status_code)

    # Normalização do arquivo em json e tranferência para um Dataframe.
    data = response_pag1.json()
    df_update = pd.json_normalize(data, record_path=['hits','hits'])

    # Verificar a data da primeira página
    if lastRecord == df_update['_source.data_importacao_rnds'][0][:10]:
        print(f'Data is already updated. Last record: {lastRecord}')

        return df
        
    else:
        # Para as próximas chamadas devemos gerar o scroll id para inserir no corpo da chamada da API
        scroll_id = data['_scroll_id']

        df = pd.concat([df,df_update],ignore_index=True)

        # Loop while para conferir os conteúdos das demais páginas onde pag representa o número da página
        pag = 1
        while lastRecord != df_update['_source.data_importacao_rnds'][0][:10]:

            pag+=1

            try:
                # Chamada das demais páginas.
                response_pagN = requests.post(url_nextpages, auth=(username, password), json={"scroll_id":scroll_id,"scroll": "1m"})
        
                if pag%10 == 0:
                    print(f'Page {pag} - Status Response: {response_pagN.status_code}')
        
                responses.append(response_pagN.status_code)
        
                # Normalização do arquivo em json e tranferência para um Dataframe.
                dataN = response_pagN.json()
                df_update = pd.json_normalize(dataN, record_path=['hits','hits'])

                print(f'Data is already updated. Last record: {lastRecord}')

                # Concatenação dos dados extraídos das seguintes leituras da API
                df = pd.concat([df,df_update],ignore_index=True)
                scroll_id = dataN['_scroll_id']

            except MemoryError:
                print(f'At Page {pag} there was not enough memory to execute the task - Status Response: {responses[-1]}\nThe scroll_id has been added to the list to a future attempt')
                scroll_ids.append(scroll_id)

            except ConnectionError:
                print('No Internet Connection to Perform the API Call. The scroll_id has been added to the list to a future attempt')
                scroll_ids.append(scroll_id)


        print(f'Data have been updated. Last record: {lastRecord}')

        print(f'{pag} Pages have been loaded successfully!')

        print(f'The following responses status have been retrieved: {set(responses)}')

        print(f'Final Dataset Shape:\nRows: {df.shape[0]}\nColumns: {df.shape[1]}')
    
        return df, responses, scroll_ids

## 1.2. Limpeza dos Dados

In [41]:
df_vac, responses, scroll_ids = getAPIData(10)

Page 1 - Status Response: 200
Page 10 - Status Response: 200
10 Pages have been loaded successfully!
The following responses status have been retrieved: {200}
Final Dataset Shape:
Rows: 100000
Columns: 46


In [16]:
for column in df_vac.columns:
    print(column)

_index
_type
_id
_score
_source.estalecimento_noFantasia
_source.vacina_lote
_source.estabelecimento_municipio_codigo
_source.estabelecimento_valor
_source.vacina_nome
_source.ds_condicao_maternal
_source.paciente_endereco_coPais
_source.@version
_source.document_id
_source.paciente_nacionalidade_enumNacionalidade
_source.vacina_categoria_codigo
_source.vacina_fabricante_referencia
_source.paciente_id
_source.paciente_idade
_source.vacina_descricao_dose
_source.paciente_endereco_coIbgeMunicipio
_source.vacina_grupoAtendimento_codigo
_source.data_importacao_datalake
_source.paciente_racaCor_codigo
_source.estabelecimento_uf
_source.estabelecimento_razaoSocial
_source.vacina_numDose
_source.@timestamp
_source.sistema_origem
_source.paciente_dataNascimento
_source.paciente_endereco_uf
_source.vacina_fabricante_nome
_source.paciente_endereco_cep
_source.id_sistema_origem
_source.status
_source.paciente_endereco_nmPais
_source.vacina_categoria_nome
_source.paciente_endereco_nmMunicipio
_sou

In [42]:
all_columns = '''_index
_type
_id
_score
_source.estalecimento_noFantasia
_source.vacina_lote
_source.estabelecimento_municipio_codigo
_source.estabelecimento_valor
_source.vacina_nome
_source.ds_condicao_maternal
_source.paciente_endereco_coPais
_source.@version
_source.document_id
_source.paciente_nacionalidade_enumNacionalidade
_source.vacina_categoria_codigo
_source.vacina_fabricante_referencia
_source.paciente_id
_source.paciente_idade
_source.vacina_descricao_dose
_source.paciente_endereco_coIbgeMunicipio
_source.vacina_grupoAtendimento_codigo
_source.data_importacao_datalake
_source.paciente_racaCor_codigo
_source.estabelecimento_uf
_source.estabelecimento_razaoSocial
_source.vacina_numDose
_source.@timestamp
_source.sistema_origem
_source.paciente_dataNascimento
_source.paciente_endereco_uf
_source.vacina_fabricante_nome
_source.paciente_endereco_cep
_source.id_sistema_origem
_source.status
_source.paciente_endereco_nmPais
_source.vacina_categoria_nome
_source.paciente_endereco_nmMunicipio
_source.estabelecimento_municipio_nome
_source.vacina_codigo
_source.paciente_enumSexoBiologico
_source.dt_deleted
_source.co_condicao_maternal
_source.vacina_grupoAtendimento_nome
_source.paciente_racaCor_valor
_source.vacina_dataAplicacao
_source.data_importacao_rnds
'''
all_columns = all_columns.split('\n')
all_columns.pop(-1)


columns = '''document_id
paciente_id
paciente_idade
paciente_dataNascimento
paciente_enumSexoBiologico
paciente_racaCor_codigo
paciente_racaCor_valor
paciente_endereco_colbgeMunicipio
paciente_endereco_coPais
paciente_endereco_nmMunicipio
paciente_endereco_nmPais
paciente_endereco_uf
paciente_endereco_cep
paciente_nacionalidade_enumNacionalidade
estabelecimento_valor
estabelecimento_razaoSocial
estalecimento_noFantasia
estabelecimento_municipio_codigo
estabelecimento_municipio_nome
estabelecimento_uf
vacina_grupo_atendimento_code
vacina_grupoAtendimento_nome
vacina_categoria_codigo
vacina_categoria_nome
vacina_lote
vacina_fabricante_nome
vacina_fabricante_referencia
vacina_dataAplicacao
vacina_descricao_dose
vacina_numDose
vacina_codigo
vacina_nome
sistema_origem
data_importacao_rnds
'''
columns = columns.split('\n')
columns.pop(-1)

for i in range(len(columns)):
    columns[i] = '_source.'+columns[i]

In [43]:
columns_toRemove = []

for column in all_columns:
    if column not in columns:
        columns_toRemove.append(column)

print(len(columns_toRemove))

14


In [44]:
df_vacRed = df_vac.drop(columns_toRemove, axis=1)

In [45]:
df_vacRed.shape

(100000, 32)

In [46]:
pd.set_option('display.max_columns', None)
df_vacRed.head(2)

,_source.estalecimento_noFantasia,_source.vacina_lote,_source.estabelecimento_municipio_codigo,_source.estabelecimento_valor,_source.vacina_nome,_source.paciente_endereco_coPais,_source.document_id,_source.paciente_nacionalidade_enumNacionalidade,_source.vacina_categoria_codigo,_source.vacina_fabricante_referencia,_source.paciente_id,_source.paciente_idade,_source.vacina_descricao_dose,_source.paciente_racaCor_codigo,_source.estabelecimento_uf,_source.estabelecimento_razaoSocial,_source.vacina_numDose,_source.sistema_origem,_source.paciente_dataNascimento,_source.paciente_endereco_uf,_source.vacina_fabricante_nome,_source.paciente_endereco_cep,_source.paciente_endereco_nmPais,_source.vacina_categoria_nome,_source.paciente_endereco_nmMunicipio,_source.estabelecimento_municipio_nome,_source.vacina_codigo,_source.paciente_enumSexoBiologico,_source.vacina_grupoAtendimento_nome,_source.paciente_racaCor_valor,_source.vacina_dataAplicacao,_source.data_importacao_rnds
0,USF DR WALDIR BUGALHO DE MEDEIROS,PCA0083,130260,2013878,COVID-19 PFIZER - COMIRNATY,10,cdbd3f76-3159-435f-a29f-00762d556b1c-i0b0,B,2,None,af23ba203c916f8de717bbf01ba55a164dfc0a52d25e15...,46.0,4ª Dose,03,AM,MANAUS SECRETARIA MUNICIPAL DE SAUDE,4,None,1976-06-27,AM,PFIZER,69088,BRASIL,Faixa Etária,MANAUS,MANAUS,87,F,Pessoas de 18 a 64 anos,PARDA,2022-11-18T00:00:00.000Z,2022-11-21T01:42:27.000Z
1,SECRETARIA MUNICIPAL DE SAUDE DE BOCAIUVA,210540,310730,6565379,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,10,fd0d520f-efb5-4340-9c94-c36aafe5eb91-i0b0,B,1,29501,6c6d1f8a8e574245e269944976e2ef64044b26ad627323...,53.0,Dose Adicional,04,MG,MUNICIPIO DE BOCAIUVA,37,Novo PNI,1968-07-29,MG,SINOVAC/BUTANTAN,39390,BRASIL,Comorbidades,BOCAIUVA,BOCAIUVA,86,F,Diabetes Mellitus,AMARELA,2021-12-02T00:00:00.000Z,2022-11-20T21:53:41.000Z


## 1.3. Exploração dos Dados

In [59]:
df_vacRed.describe()

,_source.paciente_idade
count,99992.000000
mean,38.899462
std,21.603099
min,0.000000
25%,23.000000
50%,38.000000
75%,53.000000
max,121.000000


In [58]:
df_vacRed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   _source.estalecimento_noFantasia                  100000 non-null  object 
 1   _source.vacina_lote                               88427 non-null   object 
 2   _source.estabelecimento_municipio_codigo          100000 non-null  object 
 3   _source.estabelecimento_valor                     100000 non-null  object 
 4   _source.vacina_nome                               100000 non-null  object 
 5   _source.paciente_endereco_coPais                  99896 non-null   object 
 6   _source.document_id                               100000 non-null  object 
 7   _source.paciente_nacionalidade_enumNacionalidade  99983 non-null   object 
 8   _source.vacina_categoria_codigo                   72015 non-null   object 
 9   _sour

In [60]:
df_vacRed['_source.vacina_lote'].value_counts()

FT5177        11628
211J21A        3764
PCA0084        3440
212J21A        2796
223VCD067W     2399
              ...  
206h21A           1
215VCV191W        1
4381413           1
210390            1
FP                1
Name: _source.vacina_lote, Length: 1836, dtype: int64

# 2. WebScraping - Dados de Óbitos

In [13]:
file_url = 'https://data.brasil.io/dataset/covid19/caso_full.csv.gz'

file = '../data/caso_full.rar'

response = requests.get(file_url)

with open(file, "wb") as casos:
    casos.write(response.content)

In [37]:
df_cases = pd.read_csv('../data/caso_full.csv',sep=',')

In [39]:
df_cases.head()

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-17,0.0,0,1,city,AC,3,0
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0,0,1,state,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-18,0.0,0,2,city,AC,0,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0,0,2,state,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,False,4,0.96754,2020-03-19,0.0,0,3,city,AC,1,0
